In [1]:
import numpy as np
from datetime import datetime


from CaliPytion.core import Calibrator, Standard

In [2]:
print(Standard)

Standard
├── id
├── species_id
├── name
├── wavelength
├── signal_type
├── samples
│   └── 0
│       ├── id
│       ├── concentration
│       ├── conc_unit
│       └── signal
├── ph
├── temperature
├── temperature_unit
├── created
└── model_result
    ├── id
    ├── name
    ├── equation
    ├── parameters
    │   └── 0
    │       ├── id
    │       ├── name
    │       ├── value
    │       ├── init_value
    │       ├── standard_error
    │       ├── lower_bound
    │       └── upper_bound
    ├── was_fitted
    ├── calibration_range
    │   ├── id
    │   ├── conc_lower
    │   ├── conc_upper
    │   ├── signal_lower
    │   └── signal_upper
    └── statistics
        ├── id
        ├── aic
        ├── bic
        ├── r2
        └── rmsd



## Generate artificial data for a `Standard`

In [3]:
def random(xs):
    return xs + np.random.normal(0, 0.03, 1)


SLOPE = 0.014
CONC = np.linspace(0, 200, 11)
ABSO = CONC * SLOPE

In [4]:
# Generate standard
standard = Standard(
    species_id="s0",
    name="ABTS",
    wavelength=340,
    ph=2,
    temperature=25,
    temperature_unit="C",
    created=datetime.now(),
)

# Add samples with noise
for conc, abso in zip(CONC, ABSO):
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )

## Initialize a `Calibrator` from a `Standard`

By defining a cutoff value, samples with a higher signal than the defined cutoff are ignored for the calibration.
For concentration calculation, the model does not extrapolate beyond the cutoff value by default.

In [5]:
# initialize calibrator
calibrator = Calibrator.from_standard(standard, cutoff=2.5)

CaliPytion contains predefined models, which can be used for calibration.
Alternatively, custom models can be added. 

```{note}
The equation of a model must include 'signal' and 'concentration' as variables.
```

In [6]:
exponential = calibrator.add_to_models(
    name="exponential",
    equation="a * exp(b * concentration) = signal",
)

By calling `fit_models`, all models are fitted to the data defined in `Standard`. A report, summarizing statistical parameters of each model is generated.

In [7]:
# Fit all defined models
calibrator.fit_models()

,AIC,R squared,RMSD
Model Name,,,
linear,-212,0.9988,0.0281
cubic,-212,0.9989,0.0262
quadratic,-210,0.9988,0.0280
exponential,-88,0.9293,0.2142


## Model visualization

Models can be visualized, displaying the measured samples used for fitting of thee model, as well as the fitted model. Additionally, the residuals of the model are visualized.

In [8]:
# Visualize the models
calibrator.visualize()

## Save model to `Standard`

After assessing different models, the best model can be saved to the `Standard` object.

In [9]:
linear = calibrator.get_model("linear")
print(linear)

CalibrationModel
├── id = calibrationmodel1
├── name = linear
├── equation = a * concentration = signal
├── parameters
│   └── 0
│       └── Parameter
│           ├── id = parameter0
│           ├── name = a
│           ├── value = 0.013937575974455005
│           ├── init_value = 0.1
│           ├── standard_error = 4.879788345767215e-05
│           ├── lower_bound = -inf
│           └── upper_bound = inf
├── was_fitted = True
├── calibration_range
│   └── CalibrationRange
│       ├── id = calibrationrange4
│       ├── conc_lower = 0.0
│       ├── conc_upper = 180.0
│       ├── signal_lower = -0.03469441725719837
│       └── signal_upper = 2.499949447019015
└── statistics
    └── FitStatistics
        ├── id = fitstatistics4
        ├── aic = -212.40948313789045
        ├── bic = -211.0082857562283
        ├── r2 = 0.9987881951991484
        └── rmsd = 0.028057722144518307



In [10]:
standard = calibrator.save_model(linear)

## Calculate concentrations

Models can be used to calculate concentrations of unknown signals. 

In [11]:
linear.calculate([1, 0.1, 2.234])

[71.74848781687825, 7.1748487816878255, 160.286121782906]

If the signal is out of calibration bonds, the calculate method returns `float('nan')` values for the respective signals.

In [12]:
linear.calculate([1, 100, 5, 0.5])

[71.74848781687825, nan, nan, 35.87424390843913]

In [13]:
standard.to_animl()

ConfigError: field "category" not yet prepared so type is still a ForwardRef, you might need to call Category.update_forward_refs().